# Melodi Python SDK - Quick Start

In [ ]:
import os

# Change the working directory
os.chdir("..")

# Verify the change
print("Current working directory:", os.getcwd())

In [ ]:
import sys
!{sys.executable} -m pip install .

In [ ]:
from melodi.melodi_client import MelodiClient
from melodi.feedback.data_models import Feedback
from melodi.threads.data_models import Message, Thread
from melodi.users.data_models import User

## Setup Melodi client

In [ ]:
os.environ["MELODI_API_KEY"] = "<YOUR_KEY>"

client = MelodiClient(verbose=True)

## Create a Project

In [ ]:
new_project = client.create_project("Melodi SDK Test")

print(new_project.id)

## Create a thread in Melodi

In [ ]:

melodiClient = MelodiClient(verbose=True)

threadExternalId = "external-thread-id-1"

externalUser = User(
    externalId="test-user-id-1",
    name="Test User",
    email="test-user@example.com",
    segments={
      "team": "engineering",
    }
)

message1 = Message(
    externalId="message1",
    role='user',
    content='Tell me a joke',
)

message2 = Message(
    externalId="message2",
    role='json_tool',
    type='json',
    jsonContent={
        "joke_setup": "Why did the chicken cross the road?",
        "joke_punchline": "To get to the other side!"
    },
    metadata={
        "metadatatest": "value2"
    },
)

message3 = Message(
    externalId="message3",
    role='assistant',
    content='Why did the chicken cross the road? To get to the other side!',
    metadata={
        "metadatatest": "value3"
    },
)

thread = Thread(
    projectName="Melodi SDK Test",
    externalId=threadExternalId,
    messages=[message1, message2, message3],
    externalUser=externalUser
)
melodi_thread = melodiClient.threads.create_or_update(thread)

print(melodi_thread)

## Add feedback to the thread

In [ ]:
feedback = Feedback(
    externalThreadId=melodi_thread.id,
    feedbackText="I don't understand",
    feedbackType="NEGATIVE"
)

melodi_feedback = melodiClient.feedback.create(feedback)